<a href="https://colab.research.google.com/github/ChitranshS/MinorProject/blob/main/ESRGAN_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation


In [1]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

  Cloning https://github.com/sberbank-ai/Real-ESRGAN.git to /tmp/pip-req-build-qi4f9yd5
  Running command git clone --filter=blob:none --quiet https://github.com/sberbank-ai/Real-ESRGAN.git /tmp/pip-req-build-qi4f9yd5
  Resolved https://github.com/sberbank-ai/Real-ESRGAN.git to commit 362a0316878f41dbdfbb23657b450c3353de5acf
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusol

# Upscale images

In [2]:
#@title Create model
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "8" #@param ["2", "4", "8"] {allow-input: false}

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:655: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RealESRGAN_x8.pth:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

Weights downloaded to: weights/RealESRGAN_x8.pth


In [4]:
#@title Upload and upscale images or .tar archives
import os
import shutil
import tarfile
from PIL import Image
import numpy as np
from google.colab import files
from io import BytesIO

upload_folder = 'inputs'
result_folder = 'results1'
result_tar_name = 'result_images.tar'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

def process_input(filename):
    if filename.endswith('.tar'):
        process_tar(filename)
    else:
        image = Image.open(filename).convert('RGB')
        result_image_path = os.path.join(result_folder, os.path.basename(filename))
        result_image_path_hr = os.path.join(result_folder, 'hr_' + os.path.basename(filename))
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        shutil.copy(filename, result_image_path_hr)

def process_tar(path_to_tar):
    with tarfile.open(path_to_tar, mode='r') as processing_tar:
        for member in processing_tar:
            if not member.name.endswith(('.png', '.jpg', '.jpeg')):
                continue

            try:
                img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
                img_lr = Image.open(img_bytes, mode='r').convert('RGB')
            except Exception as err:
                print(f'Unable to open file {member.name}, skipping')
                continue

            sr_image = model.predict(np.array(img_lr))
            result_image_path = os.path.join(result_folder, member.name)
            sr_image.save(result_image_path)

def create_tar_archive():
    shutil.make_archive(result_folder, 'zip', result_folder)

# Upload files
uploaded = files.upload()

# Process uploaded files
for filename in uploaded.keys():
    print('Processing:', filename)
    uploaded_path = os.path.join(upload_folder, filename)
    with open(uploaded_path, 'wb') as f:
        f.write(uploaded[filename])
    process_input(uploaded_path)

# Create tar archive
create_tar_archive()

# Download result
files.download(result_folder + '.zip')


Saving 724_55.jpg to 724_55.jpg
Saving 724_61.jpg to 724_61.jpg
Saving 659_10.jpg to 659_10.jpg
Saving 532_15.jpg to 532_15.jpg
Saving 459_5.jpg to 459_5.jpg
Saving 331_14.jpg to 331_14.jpg
Saving 148_544.jpg to 148_544.jpg
Saving 66_125.jpg to 66_125.jpg
Saving 66_36.jpg to 66_36.jpg
Saving 88_385.jpg to 88_385.jpg
Saving 21_16.jpg to 21_16.jpg
Saving 29_28.jpg to 29_28.jpg
Saving 30_24.jpg to 30_24.jpg
Processing: 724_55.jpg
Processing: 724_61.jpg
Processing: 659_10.jpg
Processing: 532_15.jpg
Processing: 459_5.jpg
Processing: 331_14.jpg
Processing: 148_544.jpg
Processing: 66_125.jpg
Processing: 66_36.jpg
Processing: 88_385.jpg
Processing: 21_16.jpg
Processing: 29_28.jpg
Processing: 30_24.jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>